# Data quality with great expectations
Some simple experiments

## Get a simple dataset

In [3]:
import pandas as pd
from sklearn.datasets import load_iris

data = load_iris()

df = pd.DataFrame(data=data.data, columns=data.feature_names)
df['target'] = data.target_names[data.target]

df = df.sample(frac=1)

df_reference = df[0:100].copy()
df_reference.to_csv("data/reference.csv")

df_test = df[100:].copy()

## Degrade the test set and see if the DQ measures detect it

In [ ]:
df_test.tail()

In [ ]:
# Put a new value in target

#df_test.at[df_test.head(1).index, 'target'] ='bob'
df_test.iloc[-1, len(df_test.columns)-1]='bob'             # target value outside expected set
df_test.iloc[-1, len(df_test.columns)-2]=123.456           # rogue big value
df_test.to_csv("data/test.csv")
df_test.tail()

# Using GE in your programs (work in progress)

In [1]:
# https://docs.greatexpectations.io/docs/guides/validation/advanced/how_to_validate_data_with_an_in_memory_checkpoint

import great_expectations as ge
from great_expectations.checkpoint import Checkpoint

from ruamel import yaml
yaml = yaml.YAML(typ="safe")

context = ge.get_context()

my_checkpoint_name = "in_memory_checkpoint"
python_config = {
    "name": my_checkpoint_name,
    "config_version": 1,
    "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
    "action_list": [
        {
            "name": "store_validation_result",
            "action": {"class_name": "StoreValidationResultAction"},
        },
        {
            "name": "store_evaluation_params",
            "action": {"class_name": "StoreEvaluationParametersAction"},
        },
        {
            "name": "update_data_docs",
            "action": {"class_name": "UpdateDataDocsAction", "site_names": []},
        },
    ],
    "validations": [
        {
            "batch_request": {
                "datasource_name": "iris",
                "data_connector_name": "default_inferred_data_connector_name",
                "data_asset_name": "test.csv",
                "data_connector_query": {"index": -1},
            },
            "expectation_suite_name": "iris3",
        }
    ],
}
my_checkpoint = Checkpoint(data_context=context, **python_config)
results = my_checkpoint.run()





DatasourceError: Cannot initialize datasource iris, error: The given datasource could not be retrieved from the DataContext; please confirm that your configuration is accurate.